In [2]:
import micromegas
import datetime
import pandas as pd
import pyarrow as pa
pd.set_option('display.max_colwidth', None)
client = micromegas.connect()

In [ ]:
sql = """
SELECT properties, count(*)
FROM measures
WHERE array_length(properties) > 0
group by properties
"""
df = client.query(sql)
df

In [ ]:
sql = """
WITH md AS(
SELECT properties_to_dict(properties) as dproperties, properties_length(properties) as nb
FROM measures
)
SELECT properties_length(dproperties) as nbtest, nb, property_get(dproperties, 'target')
FROM md
"""
print("sql:", sql)
df = client.query(sql)
assert( (df["nbtest"] == df["nb"]).all() )
display(df)

In [4]:
def inspect_dataframe_schema(df):
      """Complete inspection of DataFrame for dictionary encoding - handles complex types"""

      print("=== DataFrame Schema Analysis ===")
      print(f"Shape: {df.shape}")
      print(f"Total memory: {df.memory_usage(deep=True).sum():,} bytes")
      print("\nColumn Analysis:")

      for col in df.columns:
          col_data = df[col]
          memory = col_data.memory_usage(deep=True)

          print(f"\nColumn: {col}")
          print(f"  Dtype: {col_data.dtype}")
          print(f"  Memory: {memory:,} bytes")

          # Modern way to check for categorical
          if isinstance(col_data.dtype, pd.CategoricalDtype):
              print(f"  ✓ Categorical with {col_data.cat.categories.size} categories")
              print(f"  Categories: {list(col_data.cat.categories)}")

          # Check for Arrow backing
          if hasattr(col_data.dtype, 'pyarrow_dtype'):
              arrow_type = col_data.dtype.pyarrow_dtype
              print(f"  ✓ Arrow-backed: {arrow_type}")
              if pa.types.is_dictionary(arrow_type):
                  print(f"  ✓ Arrow Dictionary: {arrow_type.index_type} -> {arrow_type.value_type}")

          # Safe unique count for complex types
          try:
              unique_count = col_data.nunique()
              total_count = len(col_data)
              print(f"  Unique values: {unique_count:,} / {total_count:,} ({unique_count/total_count:.1%})")
          except (TypeError, ValueError) as e:
              print(f"  Unique count: Cannot compute (complex data type: {e.__class__.__name__})")
              total_count = len(col_data)
              print(f"  Total rows: {total_count:,}")

              # Try to inspect first few values
              try:
                  sample = col_data.iloc[0]
                  print(f"  Sample value type: {type(sample)}")
                  if hasattr(sample, 'shape'):
                      print(f"  Sample shape: {sample.shape}")
              except Exception as e2:
                  print(f"  Cannot inspect sample: {e2.__class__.__name__}")

In [5]:
sql = "SELECT properties_to_dict(properties) as properties FROM measures"
print("sql:", sql)
rbs = list(client.query_stream(sql))
table = pa.Table.from_batches(rbs)
print("schema:", table.schema)
inspect_dataframe_schema(table.to_pandas())

sql: SELECT properties_to_dict(properties) as properties FROM measures
schema: properties: list<Property: struct<key: string not null, value: string not null> not null>
  child 0, Property: struct<key: string not null, value: string not null> not null
      child 0, key: string not null
      child 1, value: string not null
=== DataFrame Schema Analysis ===
Shape: (9380319, 1)
Total memory: 1,125,638,412 bytes

Column Analysis:

Column: properties
  Dtype: object
  Memory: 1,125,638,412 bytes
  Unique count: Cannot compute (complex data type: TypeError)
  Total rows: 9,380,319
  Sample value type: <class 'numpy.ndarray'>
  Sample shape: (0,)
